In [30]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os

In [38]:
base_folder = 'hot'

# Resize transform
resize = transforms.Resize((128, 128))

In [39]:
def calculate_miou(inference_folder, target_folder, output_folder):
    # Initialize variables for mIoU calculation
    total_iou = 0.0
    num_images = 0

    # Process each pair of images
    for filename in os.listdir(inference_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            inference_image_path = os.path.join(inference_folder, filename)
            target_image_path = os.path.join(target_folder, filename)

            if not os.path.exists(target_image_path):
                print(f"Target image not found for {filename}, skipping.")
                continue

            # Load images
            inference_image = Image.open(inference_image_path).convert('L')
            target_image = Image.open(target_image_path).convert('L')

            # Resize images to 128x128
            inference_image_resized = resize(inference_image)
            target_image_resized = resize(target_image)

            # Convert images to binary masks (thresholding)
            inference_image_binary = np.array(inference_image_resized) > 128
            target_image_binary = np.array(target_image_resized) > 128

            # Convert binary masks to tensors
            inference_tensor = torch.tensor(inference_image_binary, dtype=torch.float32)
            target_tensor = torch.tensor(target_image_binary, dtype=torch.float32)

            # Calculate Intersection over Union (IoU)
            intersection = torch.sum(inference_tensor * target_tensor)
            union = torch.sum(inference_tensor) + torch.sum(target_tensor) - intersection
            iou = intersection / union

            # Accumulate IoU for mIoU calculation
            total_iou += iou.item()
            num_images += 1

            # Create an RGB image to visualize overlap
            overlap_image = np.zeros((128, 128, 3), dtype=np.uint8)

            # Red for inference image
            overlap_image[inference_image_binary, 0] = 255

            # Blue for target image
            overlap_image[target_image_binary, 2] = 255

            # Purple for overlap
            overlap_image[inference_image_binary & target_image_binary] = [255, 0, 255]

            # Convert to PIL image and save
            overlap_image_pil = Image.fromarray(overlap_image)
            overlap_image_path = os.path.join(output_folder, f'overlap_{filename}')
            overlap_image_pil.save(overlap_image_path)

            print(f'{filename} - IoU: {iou.item()}')
            print(f'Overlap image saved to {overlap_image_path}')

    # Calculate mean IoU (mIoU)
    if num_images > 0:
        miou = total_iou / num_images
        print(f'Mean IoU (mIoU) for all images: {miou}')
    else:
        print('No images processed.')
        miou = 0.0

    return miou

In [ ]:
# Dynamically find all subfolders in the base folder
sub_folders = [f.name for f in os.scandir(base_folder) if f.is_dir()]

# Process each subfolder
for sub_folder in sub_folders:
    print(f'Processing {sub_folder}...')

    inference_folder = os.path.join(base_folder, sub_folder, 'inference')
    rule_inference_folder = os.path.join(base_folder, sub_folder, 'rule_inference')
    target_folder = os.path.join(base_folder, sub_folder, 'target')
    output_folder_inference = os.path.join(base_folder, sub_folder, 'output_inference')
    output_folder_rule_inference = os.path.join(base_folder, sub_folder, 'output_rule_inference')

    os.makedirs(output_folder_inference, exist_ok=True)
    os.makedirs(output_folder_rule_inference, exist_ok=True)

    # Calculate mIoU for inference vs target
    miou_inference = calculate_miou(inference_folder, target_folder, output_folder_inference)
    print(f'{sub_folder} - mIoU for inference vs target: {miou_inference}')

    # Calculate mIoU for rule_inference vs target
    miou_rule_inference = calculate_miou(rule_inference_folder, target_folder, output_folder_rule_inference)
    print(f'{sub_folder} - mIoU for rule_inference vs target: {miou_rule_inference}')